# Issues with 1.) units of mass when adding phases to system and 2.) formula parser 

I will illustrate issue 1 by comparing the amount of Fe added to a chemical system and the amount of Fe subsequently reported to be in that system. I will illustrate issue two by comparing the molar masses of the same substance but with different formula formats. 

## 1. Import Libraries

In [78]:
import thermofun as fun
import reaktoro as rkt
import numpy as np
import matplotlib.pyplot as plt

## 2. Initialize a thermodynamic database

In [79]:
database = fun.Database('./databases/Fe-O_system.json')

### 2a. Check the contents of the database (optional)

In [80]:
substances = database.mapSubstances()
print(substances.keys())

dict_keys(['Hematite', 'Magnetite', 'Magnetite_charge_specified', 'O2'])


## 3. Define and construct the chemical system

In [81]:
# Define the chemical system
editor = rkt.ChemicalEditor(database)
editor.addMineralPhase(['Hematite', 'Magnetite']) 
editor.addGaseousPhase(['O2'])

# Construct the chemical system
system = rkt.ChemicalSystem(editor)

## 4. Define and solve the equilibrium problem

### 4a. Set mass of hematite and magnetite to be added to system; calculate total moles of Fe to be added

In [90]:
# mass of hematite and magnetite to be added
mass_hematite = 1 # [g]
mass_magnetite = 1 # [g]

# determine molar mass of hematite and magnetite
molar_mass_hematite = system.species()[1].molarMass() # [g*mol^-1]
molar_mass_magnetite = system.species()[2].molarMass() # [g*mol^-1]
print('The molar mass of hematite is ' + str(molar_mass_hematite) + 
      ' g/mol and the molar mass of magnetite is ' + str(molar_mass_magnetite) + ' g/mol.')

# calculate moles of hematite and magnetite added to the system
moles_hematite = mass_hematite/molar_mass_hematite # [mol]
moles_magnetite = mass_magnetite/molar_mass_magnetite # [mol]

#calculate total Fe added to the system
moles_Fe_added = moles_hematite*2 + moles_magnetite*3
print(str(moles_Fe_added) + ' moles of Fe will be added to the system.')

The molar mass of hematite is 159.6882028579712 g/mol and the molar mass of magnetite is 231.5326042175293 g/mol.
0.025481545127003125 moles of Fe will be added to the system.


### 4b. Add phases to system; solve equilibrium state

In [83]:
problem = rkt.EquilibriumProblem(system)

# add phases to system
problem.add("Hematite",mass_hematite,"g") 
problem.add("Magnetite",mass_magnetite,"g") 
problem.add("O2",10**-2,"mol")

# set T,P
problem.setPressure(1,"bar")
problem.setTemperature(500,"kelvin")

# determine equilibrium state
solver = rkt.EquilibriumSolver(system)
state = rkt.ChemicalState(system)
solver.solve(state, problem)    

#state.output

### 4c. Determine amount of Fe reported to be in the system

In [84]:
# determine mole amount Fe reported to be in the system
moles_Fe_reported = state.elementAmount("Fe") # [mol]

print(str(moles_Fe_reported) + ' moles of Fe reported to be in the system.')

2.5481545127003124e-05 moles of Fe reported to be in the system.


Note the discrepancy between the moles of Fe added to the system (2.5E-2) and the moles of Fe reported to be in the system (2.5E-5). The 10^3 difference between the two can be eliminated by setting the units in, for example, `problem.add("Hematite",mass_hematite,"g")` to `kg` (i.e., `problem.add("Hematite",mass_hematite,"kg")`. This is of course inaccurate. 

## 5. Issue with formula parser when evaluating magnetite formula

Furthermore, there is an issue with the formula parser for magnetite (it's not clear if this is a problem with `Reaktoro` or `ThermoFun`). For the entry "Magnetite," the formula is given as "Fe3O4." For the entry "Magnetite_charge_specified," the formula is given as "FeFe|3|2O4." The latter corresponds with the original entry in the mines16-thermofun.json data file; the former is the version I've corrected (both have been pasted into the data file for this notebook). Only the former ("Fe3O4") yields the correct molar mass, as can be seen below: 

In [85]:
# Define the chemical system
editor1 = rkt.ChemicalEditor(database)
editor1.addMineralPhase(['Magnetite', 'Magnetite_charge_specified']) 

# Construct the chemical system
system1 = rkt.ChemicalSystem(editor1)

# Determine molar mass of each
print('The molar mass of "Magnetite" is ' + str(system1.species()[0].molarMass()) + ' g*mol^-1. ') # [g*mol^-1]
print('The molar mass of "Magnetite_charge_specified" is ' + str(system1.species()[1].molarMass()) + ' g*mol^-1. ') # [g*mol^-1]


The molar mass of "Magnetite" is 231.5326042175293 g*mol^-1. 
The molar mass of "Magnetite_charge_specified" is 119.8426017761231 g*mol^-1. 
